In [1]:
import pandas as pd

In [2]:
desc_df = pd.read_excel("../data/desc.xlsx")

In [3]:
desc_df.head()

,project_id,project_address,desc_total,video_desc,risk_desc
0,804380059,/projects/1552697291/ufactory-3d-printing-shop/,Follow us on Facebook and Twitter 횂혻and pl...,my name is Matt Elkins not here to tell you a...,Risks and challenges The biggest challenge of...
1,1560420632,/projects/941562770/nervous-dating-dating-for-...,"""I get very nervous when I think someone is f...",introverts the idea of going on the site can ...,Risks and challenges Fulfilment of your 'free...
2,278626358,/projects/703843051/messenger-rock/,"Hi, do you feel an inner hatred that you wish...",NaN,Risks and challenges The rock quality. Learn ...
3,907761690,/projects/1407693875/wind-turbine-phone-charge...,The goal of my project is to build a wind tur...,NaN,Risks and challenges The risks and challenges...
4,1512099429,/projects/358359447/crash-surveillance-drone-csd/,The project will consist of building a protot...,NaN,Risks and challenges The initial project is t...


# Risk Perception

In [4]:
desc_df.head()

,project_id,project_address,desc_total,video_desc,risk_desc
0,804380059,/projects/1552697291/ufactory-3d-printing-shop/,Follow us on Facebook and Twitter 횂혻and pl...,my name is Matt Elkins not here to tell you a...,Risks and challenges The biggest challenge of...
1,1560420632,/projects/941562770/nervous-dating-dating-for-...,"""I get very nervous when I think someone is f...",introverts the idea of going on the site can ...,Risks and challenges Fulfilment of your 'free...
2,278626358,/projects/703843051/messenger-rock/,"Hi, do you feel an inner hatred that you wish...",NaN,Risks and challenges The rock quality. Learn ...
3,907761690,/projects/1407693875/wind-turbine-phone-charge...,The goal of my project is to build a wind tur...,NaN,Risks and challenges The risks and challenges...
4,1512099429,/projects/358359447/crash-surveillance-drone-csd/,The project will consist of building a protot...,NaN,Risks and challenges The initial project is t...


In [11]:
risk_desc_df = desc_df[['project_id', 'project_address']]

## Request

In [6]:
risk_desc_df = pd.read_pickle('risk_desc_df.pkl')

In [7]:
from selenium import webdriver

browser = webdriver.Chrome('chromedriver')

In [ ]:
risk_desc_list = []

for i, r in risk_desc_df.iterrows():
    print(i)
    if len(r['text'])==0:
        try:
            url = 'https://www.kickstarter.com'+r['project_address']
            browser.get(url)  
            browser.set_page_load_timeout(45)
            browser.implicitly_wait(10)

            text= browser.find_element_by_class_name('js-risks-text').text
            risk_desc_list.append(text)
        except:
            risk_desc_list.append('')
    else:
        risk_desc_list.append(r['text'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173


In [ ]:
browser.close()

In [ ]:
risk_desc_df.shape

In [ ]:
risk_desc_df['text'] = risk_desc_list

In [ ]:
risk_desc_df.to_pickle('risk_desc_df.pkl')

In [ ]:
risk_desc_df.to_csv('risk_desc_df.csv', encoding='utf-8')

In [ ]:
risk_desc_df.to_excel('risk_desc_df.xlsx', encoding='utf-8')